In [14]:
from keras.models import Model, load_model, Sequential

from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from random import shuffle
#from scipy.misc import imresize
import itertools
import matplotlib.pyplot as plt

import os
%matplotlib inline

#from keras import models , optimizers , losses ,activations , callbacks

import keras.backend as K
from keras import Sequential

In [16]:
DATA_DIR = r'd:\ML\data\f'
IMAGE_DIR  = os.path.join(DATA_DIR, 'images', "byteam")
VAL_DATA_DIR = os.path.join(DATA_DIR, 'images', "byteam_validation")

In [39]:
imageStandardizer=ImageStandardizer(MAX_THUMB_SIZE)

In [9]:
from keras.models import Model, load_model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers , losses , activations , callbacks

HW_SIZE=(80,32)
MAX_THUMB_SIZE=(32,80)
NUM_TEAMS = 4

from ImageStandardizer import ImageStandardizer
class TeamRecognizer :

    def __init__( self, max_thumb_size, model=None ):
        self.max_thumb_size = max_thumb_size
        self.input_shape = (self.max_thumb_size[1], self.max_thumb_size[0], 3)    
        self.imageStandardizer = ImageStandardizer(self.max_thumb_size)
        self.class_indices = {}
        
        if model:
            self._model = model
            #TODO: 
            return
        
        self._model = self.basic_net()

        self._model.compile( loss=losses.binary_crossentropy , optimizer='rmsprop', metrics=['accuracy'])
        #self._model.compile(optimizer='sgd', loss=losses.binary_crossentropy, metrics=['accuracy'])
    def basic_net(self):
        cnn = Sequential()
        cnn.add(Conv2D(filters=32, 
                       kernel_size=(2,2), 
                       strides=(1,1),
                       padding='same',
                       input_shape=  self.input_shape,
                       data_format='channels_last'))
        cnn.add(Activation('relu'))
        cnn.add(MaxPooling2D(pool_size=(2,2),
                             strides=2))
        cnn.add(Conv2D(filters=64,
                       kernel_size=(2,2),
                       strides=(1,1),
                       padding='valid'))
        cnn.add(Activation('relu'))
        cnn.add(MaxPooling2D(pool_size=(2,2), strides=2))
        cnn.add(Flatten())        
        cnn.add(Dense(64))
        cnn.add(Activation('relu'))
        cnn.add(Dropout(0.1))
        cnn.add(Dense(NUM_TEAMS))
        cnn.add(Activation('softmax'))     
        return cnn
    
    def fit_generator(self, train_generator = None , validation_datagen=None, epochs=10, **kwargs  ):
        if not train_generator:
            train_datagen = ImageDataGenerator(
                rotation_range = 10,                  
                width_shift_range = 2,                  
                height_shift_range = 2,                  
                rescale = 1./255,                                   
                zoom_range = 0.01,                     
                horizontal_flip = True)
            train_generator = train_datagen.flow_from_directory(
                IMAGE_DIR,
                target_size=HW_SIZE,
                class_mode='categorical',
                batch_size = BATCH_SIZE)
            
        if not validation_datagen:
            validation_datagen = ImageDataGenerator(rescale = 1./255)    
            validation_generator = validation_datagen.flow_from_directory(
                VAL_DATA_DIR,
                target_size=HW_SIZE,
                class_mode='categorical',
                batch_size = BATCH_SIZE)
        import time
        start = time.time()
        cnn.fit_generator(
            train_generator,
            steps_per_epoch=NB_TRAIN_IMG//BATCH_SIZE,
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=NB_VALID_IMG//BATCH_SIZE)
        end = time.time()
        print('Fiiting time:',(end - start)/60)
    def predict(self, crop_images):
        shape = (len(crop_images),) + HW_SIZE +(3,)
        res = np.empty(shape)
        i=0
        for crop_image, box in crop_images:
            res[i] = self.imageStandardizer.resize_array(crop_image)/255
            i+=1
            
        return np.argmax(self._model.predict(res), axis=-1)
            
    def save_model(self , file_path ='team_model.h5'):
        self._model.save(file_path )
    def load_model(self , file_path ='team_model.h5'):
        self._model = models.load_model(file_path)
        
  

In [10]:
teamRecognizer = TeamRecognizer(MAX_THUMB_SIZE)    

In [48]:
import pickle
with open('crop_images.pkl', 'rb') as file:
    crop_images = pickle.load(file)

(8, 80, 32, 3)

In [50]:
shape = (len(crop_images),) + HW_SIZE +(3,)
shape
res = np.empty(shape)
i=0
for crop_image, box in crop_images:
    res[i] = imageStandardizer.resize_array(crop_image)/255
    i+=1
np.argmax(cnn.predict(res), axis=-1)    

array([2, 1, 1, 1, 0, 0, 0, 2], dtype=int64)

In [18]:
input_shape = (80,32,3)

In [57]:
cnn = Sequential()
cnn.add(Conv2D(filters=32, 
               kernel_size=(2,2), 
               strides=(1,1),
               padding='same',
               input_shape=  input_shape,
               data_format='channels_last'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2),
                     strides=2))
cnn.add(Conv2D(filters=64,
               kernel_size=(2,2),
               strides=(1,1),
               padding='valid'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2), strides=2))
cnn.add(Flatten())        
cnn.add(Dense(64))
cnn.add(Activation('relu'))
cnn.add(Dropout(0.1))
cnn.add(Dense(NUM_TEAMS))
cnn.add(Activation('softmax'))
cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [53]:
BATCH_SIZE = 32
VAL_BATCH_SIZE = 32
NUM_EPOCHS = 1
BEST_MODEL_FILE = os.path.join(DATA_DIR, "models", "best.h5")
FINAL_MODEL_FILE  = os.path.join(DATA_DIR, "models", "final.h5")
CHECKPOINT = os.path.join(DATA_DIR, "models", "model.{epoch:03d}-{loss:.4f}-{accuracy:.3f}-{val_loss:.4f}-{val_accuracy:.3f}.h5")
LATEST_MODEL = 'm3_latest.h5'

In [30]:
NB_CHANNELS =  3 #for RGB images or 1 for grayscale images

NB_TRAIN_IMG = 600# Replace with the total number training images
NB_VALID_IMG = 90# Replace with the total number validation images

In [16]:
#Prepare validation set
i=0
import shutil
sub_dirs = [f.name for f in os.scandir(IMAGE_DIR) if f.is_dir()]
for sd in sub_dirs:
    if not os.path.exists(os.path.join(VAL_DATA_DIR, sd)):
        os.makedirs(os.path.join(VAL_DATA_DIR, sd))
    fns =  [f.name for f in os.scandir(os.path.join(IMAGE_DIR, sd)) if f.is_file()]
    for fn in fns:
        if i % 5 == 0:
            shutil.move(os.path.join(IMAGE_DIR, sd, fn), os.path.join(VAL_DATA_DIR, sd, fn))
        i+=1

In [60]:
train_datagen = ImageDataGenerator(
    rotation_range = 20,                  
    width_shift_range = 3,                  
    height_shift_range = 3,                  
    rescale = 1./255,                                   
    zoom_range = 0.02,                     
    horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(
    IMAGE_DIR,
    target_size=HW_SIZE,
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    save_to_dir = 'tmp')
validation_generator = validation_datagen.flow_from_directory(
    VAL_DATA_DIR,
    target_size=HW_SIZE,
    class_mode='categorical',
    batch_size = BATCH_SIZE)

Found 606 images belonging to 4 classes.
Found 91 images belonging to 4 classes.


In [22]:
next(validation_generator)

(array([[[[0.45098042, 0.18823531, 0.12156864],
          [0.45098042, 0.18039216, 0.12156864],
          [0.45098042, 0.18431373, 0.14901961],
          ...,
          [0.41176474, 0.20784315, 0.15294118],
          [0.41960788, 0.19215688, 0.13725491],
          [0.42352945, 0.19215688, 0.13725491]],
 
         [[0.44705886, 0.18039216, 0.13725491],
          [0.44705886, 0.18431373, 0.14901961],
          [0.4431373 , 0.18431373, 0.12156864],
          ...,
          [0.40784317, 0.24705884, 0.3019608 ],
          [0.4039216 , 0.23137257, 0.18823531],
          [0.4039216 , 0.21960786, 0.14901961]],
 
         [[0.45098042, 0.18431373, 0.10980393],
          [0.4431373 , 0.18823531, 0.11764707],
          [0.43137258, 0.19215688, 0.11764707],
          ...,
          [0.3921569 , 0.21960786, 0.427451  ],
          [0.38823533, 0.22352943, 0.3647059 ],
          [0.38431376, 0.23137257, 0.30588236]],
 
         ...,
 
         [[0.47058827, 0.28627452, 0.2627451 ],
          [0.47058

In [61]:
import time
start = time.time()
cnn.fit_generator(
    train_generator,
    steps_per_epoch=NB_TRAIN_IMG//BATCH_SIZE,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=NB_VALID_IMG//BATCH_SIZE)
end = time.time()
print('Processing time:',(end - start)/60)
#cnn.save_weights('cnn_baseline.h5')

Epoch 1/15
18/18 [==============================] - 3s 179ms/step - loss: 0.0350 - accuracy: 0.9882 - val_loss: 0.0673 - val_accuracy: 0.9688
Epoch 2/15
18/18 [==============================] - 4s 195ms/step - loss: 0.0268 - accuracy: 0.9922 - val_loss: 0.0682 - val_accuracy: 0.9746
Epoch 3/15
18/18 [==============================] - 3s 181ms/step - loss: 0.0227 - accuracy: 0.9922 - val_loss: 0.1403 - val_accuracy: 0.9831
Epoch 4/15
18/18 [==============================] - 3s 174ms/step - loss: 0.0346 - accuracy: 0.9874 - val_loss: 0.0352 - val_accuracy: 0.9922
Epoch 5/15
18/18 [==============================] - 3s 167ms/step - loss: 0.0165 - accuracy: 0.9939 - val_loss: 0.0084 - val_accuracy: 0.9831
Epoch 6/15
18/18 [==============================] - 3s 174ms/step - loss: 0.0166 - accuracy: 0.9952 - val_loss: 0.0042 - val_accuracy: 0.9746
Epoch 7/15
18/18 [==============================] - 3s 190ms/step - loss: 0.0208 - accuracy: 0.9926 - val_loss: 0.0041 - val_accuracy: 0.9844
Epoch 

In [20]:
cnn = load_model('team_model.h5')

In [62]:
cnn.save('team_model.h5')

In [63]:
cnn.get_weights()

[array([[[[-1.44515261e-01,  1.47225574e-01,  1.78702399e-01,
            1.31426126e-01,  6.34747893e-02,  5.29974364e-02,
           -1.48498759e-01, -1.01666927e-01,  1.44753128e-01,
           -4.02478054e-02, -8.90702829e-02,  3.63353384e-03,
            3.74864303e-02, -4.68446687e-02, -1.01884738e-01,
           -6.31686226e-02, -1.51013777e-01,  1.79762706e-01,
            1.67658716e-01, -1.77949131e-01, -5.78337312e-02,
            1.67156741e-01, -1.80201605e-01,  1.69970050e-01,
            8.96030758e-03,  1.60756335e-01,  1.70367911e-01,
            2.01581120e-01,  1.21314637e-02, -1.32486209e-01,
            1.33351371e-01, -7.63281658e-02],
          [ 3.54681686e-02, -1.30560622e-01,  4.01193835e-02,
           -1.06745310e-01,  9.52578485e-02,  1.53673708e-01,
            4.20607142e-02, -1.23532444e-01, -1.64253935e-01,
            1.76896140e-01,  1.16851389e-01, -1.01339728e-01,
            5.10294549e-02, -1.21721998e-02,  2.04884082e-01,
            8.55887234e-

In [53]:
test_image_path = r'D:\ML\data\f\images\out\Дмитрий Пятницкий _8\VID_20200217_185954_084ed1.jpg'
test_array = ImageStandardizer(MAX_THUMB_SIZE).array_from_file(test_image_path)/255
recognizing = cnn.predict(np.expand_dims(test_array, axis=0))
recognizing

array([[1.5922684e-06, 7.3489551e-05, 7.7664308e-02, 9.2226064e-01]],
      dtype=float32)

In [46]:
np.argmax(recognizing, axis=-1)

array([2], dtype=int64)

In [64]:
train_generator.class_indices

{'01_purple': 0, '02_green': 1, '03_white': 2, '04_colored': 3}

In [66]:
class_indices_list =['01_purple', '02_green', '03_white', '04_colored']